In [3]:
pwd

'd:\\project\\machine_failuire'

In [2]:
import os 
os.chdir("..")

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df=pd.read_csv("data\data.csv")
df

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
0,0,7,7,1,6,6,36,3,1,1
1,190,1,3,3,5,1,20,4,1,0
2,31,7,2,2,6,1,24,6,1,0
3,83,4,3,4,5,1,28,6,1,0
4,640,7,5,6,4,0,68,6,1,0
...,...,...,...,...,...,...,...,...,...,...
939,0,7,7,1,6,4,73,6,24,1
940,0,7,5,2,6,6,50,6,24,1
941,0,3,6,2,7,5,43,6,24,1
942,0,6,6,2,5,6,46,7,24,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   footfall     944 non-null    int64
 1   tempMode     944 non-null    int64
 2   AQ           944 non-null    int64
 3   USS          944 non-null    int64
 4   CS           944 non-null    int64
 5   VOC          944 non-null    int64
 6   RP           944 non-null    int64
 7   IP           944 non-null    int64
 8   Temperature  944 non-null    int64
 9   fail         944 non-null    int64
dtypes: int64(10)
memory usage: 73.9 KB


In [6]:
df.describe()

,footfall,tempMode,AQ,USS,CS,VOC,RP,IP,Temperature,fail
count,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000,944.000000
mean,306.381356,3.727754,4.325212,2.939619,5.394068,2.842161,47.043432,4.565678,16.331568,0.416314
std,1082.606745,2.677235,1.438436,1.383725,1.269349,2.273337,16.423130,1.599287,5.974781,0.493208
min,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,19.000000,1.000000,1.000000,0.000000
25%,1.000000,1.000000,3.000000,2.000000,5.000000,1.000000,34.000000,3.000000,14.000000,0.000000
50%,22.000000,3.000000,4.000000,3.000000,6.000000,2.000000,44.000000,4.000000,17.000000,0.000000
75%,110.000000,7.000000,6.000000,4.000000,6.000000,5.000000,58.000000,6.000000,21.000000,1.000000
max,7300.000000,7.000000,7.000000,7.000000,7.000000,6.000000,91.000000,7.000000,24.000000,1.000000


In [7]:
X=df.drop(["fail"],axis=1)
y=df.fail

In [8]:
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn 
from torch.utils.data import DataLoader ,TensorDataset
import mlflow
import mlflow.pytorch
import optuna

d:\project\machine_failuire\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
scaler=StandardScaler()

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [11]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled =pd.DataFrame(X_train_scaled)
X_test_scaled =pd.DataFrame(X_test_scaled)
X_train_scaled

,0,1,2,3,4,5,6,7,8
0,-0.267330,-0.290470,1.872949,-1.419633,0.475704,0.987989,-1.147444,-0.359306,-0.371991
1,-0.287191,0.839968,-0.215790,0.022928,0.475704,-0.344032,2.646396,-2.260771,-1.861487
2,-0.287191,0.839968,0.480456,0.022928,0.475704,0.987989,1.020465,-1.626949,-0.206491
3,-0.279066,1.216780,0.480456,-1.419633,-1.079085,0.987989,1.923760,1.542159,-0.040991
4,-0.287191,1.216780,0.480456,-0.698353,-1.079085,0.543982,0.297828,-0.993127,0.124508
...,...,...,...,...,...,...,...,...,...
750,0.064903,1.216780,-0.215790,1.465489,-1.856479,-0.788039,-0.244149,-0.993127,-1.530488
751,-0.261010,-1.044095,0.480456,-0.698353,-1.079085,-0.344032,-1.328104,0.274516,-0.371991
752,-0.266427,0.463155,1.176702,0.022928,0.475704,1.431996,0.117169,0.908338,1.117506
753,-0.187883,1.216780,-0.215790,-0.698353,0.475704,0.543982,1.743101,-0.359306,0.124508


In [14]:
X_train_scaled = torch.tensor(X_train_scaled.values,dtype=torch.float32)
X_test_scaled = torch.tensor(X_test_scaled.values,dtype=torch.float32)
y_train = torch.tensor(y_train.values,dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values,dtype=torch.float32).unsqueeze(1)

In [13]:
import pickle

with open("data/scaler.pkl","wb") as f:
    pickle.dump(scaler,f)
    
X.columns

Index(['footfall', 'tempMode', 'AQ', 'USS', 'CS', 'VOC', 'RP', 'IP',
       'Temperature'],
      dtype='object')

In [15]:
train_ds=TensorDataset(X_train_scaled,y_train)
test_ds=TensorDataset(X_test_scaled,y_test)

In [16]:
trainloader=DataLoader(train_ds)

In [17]:
class MYNN(nn.Module):
    def __init__(self, hidden, dropout):
        super().__init__()
        self.net= nn.Sequential(
            nn.Linear(X_train_scaled.shape[1],hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.net(x)

In [18]:
from tqdm import tqdm

In [ ]:
def train_model(params):

    model = MYNN(params["hidden"], params["dropout"])
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])

   
    for epoch in range(params["epochs"]):
        model.train()
        for xb, yb in tqdm(trainloader,total=len(trainloader)):
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        test_pred = model(X_test_scaled)
        test_loss = loss_fn(test_pred,y_test)
        test_acc = ((test_pred > 0.5).float()==y_test).float().mean().item()

    return model,test_acc,test_loss

In [20]:
import mlflow.pytorch


def objective(trail):

    params={
        "hidden" : trail.suggest_int("hidden",16,64),
        "dropout" : round(trail.suggest_float("dropout",0.2,0.5),1),
        "lr" : round(trail.suggest_float("lr",0.01,0.1),2),
        "epochs" : trail.suggest_int("epochs",5,15)
    }

    with mlflow.start_run():

        model,test_acc,test_loss = train_model(params)
        
        mlflow.log_params(params)
        mlflow.log_metric("accuracy",test_acc)
        mlflow.pytorch.log_model(model)
    return test_loss



In [21]:
mlflow.set_experiment("machine_failuire")
study =optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=20)

[I 2025-07-09 14:10:44,350] A new study created in memory with name: no-name-05a428a4-c201-4ce0-aef3-fabf3a35fd41
2025/07/09 14:10:45 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRE